In [1]:
from rrdp_tools.reconstruct import http_get_delta_or_snapshot
from rrdp_tools.rrdp import parse_snapshot_or_delta, parse_notification_file

import pandas as pd
import dataclasses
import requests

In [2]:
notification = parse_notification_file(requests.get("https://rrdp.ripe.net/notification.xml").text)

In [5]:
notification.snapshot.uri

'https://rrdp.ripe.net/f351ea8a-62f5-4138-ad36-e93aa102fa88/31488/snapshot-9e16067a62a27c483ef9e4199525da8e8cd179ed87d0c268496bd7366b0621df.xml'

In [6]:
snapshot = http_get_delta_or_snapshot(notification.snapshot.uri)

In [7]:
#next(iter(parse_snapshot_or_delta(snapshot)))

In [8]:
df = pd.DataFrame.from_dict([
    dataclasses.asdict(elem) for elem in parse_snapshot_or_delta(snapshot)
])

In [9]:
df["num_slashes"] = df.uri.apply(lambda uri: len(uri.split('/')))
df["extension"] = df.uri.apply(lambda uri: uri.split('.')[-1])
df["size"] = df.content.apply(len)

In [10]:
df[['num_slashes', 'extension', 'size']].groupby(['num_slashes', 'extension']).sum()

size
num_slashes extension          
5           cer            2297
            crl             555
            mft            1786
6           cer        28788676
            crl          845189
            mft         1395935
9           crl        13242932
            mft        40216248
            roa        64114376

In [11]:
print(df[['num_slashes', 'extension', 'size']].groupby(['num_slashes', 'extension']).sum().reset_index().to_markdown())

|    |   num_slashes | extension   |     size |
|---:|--------------:|:------------|---------:|
|  0 |             5 | cer         |     2297 |
|  1 |             5 | crl         |      555 |
|  2 |             5 | mft         |     1786 |
|  3 |             6 | cer         | 28788676 |
|  4 |             6 | crl         |   845189 |
|  5 |             6 | mft         |  1395935 |
|  6 |             9 | crl         | 13242932 |
|  7 |             9 | mft         | 40216248 |
|  8 |             9 | roa         | 64114376 |


In [12]:
df[['num_slashes', 'extension', 'size']].groupby(['num_slashes', 'extension']).sum()

size
num_slashes extension          
5           cer            2297
            crl             555
            mft            1786
6           cer        28788676
            crl          845189
            mft         1395935
9           crl        13242932
            mft        40216248
            roa        64114376

In [14]:
print(df[['num_slashes', 'extension', 'size']].groupby(['num_slashes', 'extension']).max().reset_index().to_markdown())

|    |   num_slashes | extension   |    size |
|---:|--------------:|:------------|--------:|
|  0 |             5 | cer         |    1259 |
|  1 |             5 | crl         |     555 |
|  2 |             5 | mft         |    1786 |
|  3 |             6 | cer         |   30967 |
|  4 |             6 | crl         |  835047 |
|  5 |             6 | mft         | 1394023 |
|  6 |             9 | crl         |  752359 |
|  7 |             9 | mft         |   26365 |
|  8 |             9 | roa         |   74004 |


In [15]:
df[['num_slashes', 'extension', 'size']].groupby(['num_slashes', 'extension']).max()

size
num_slashes extension         
5           cer           1259
            crl            555
            mft           1786
6           cer          30967
            crl         835047
            mft        1394023
9           crl         752359
            mft          26365
            roa          74004

In [20]:
df[df['size'] > 100000]

,uri,hash,content,h_content,num_slashes,extension,size
30059,rsync://rpki.ripe.net/repository/DEFAULT/5e/79...,None,b'0\x83\x0bz\xe20\x83\x0by\xc9\x02\x01\x010\r\...,ca240c09d41373202849d2312865561d2358c4ae2aa5e9...,9,crl,752359
83602,rsync://rpki.ripe.net/repository/DEFAULT/KpSo3...,None,b'0\x83\x0c\xbd\xe20\x83\x0c\xbc\xc9\x02\x01\x...,37b32cde0f91f7f8dba107367f0d816b2e773d9cbca1d3...,6,crl,835047
83603,rsync://rpki.ripe.net/repository/DEFAULT/KpSo3...,None,b'0\x83\x15Eb\x06\t*\x86H\x86\xf7\r\x01\x07\x0...,5e64a62715b9b4ab5b97ea5ace58cbc9f331334c6dd33f...,6,mft,1394023


In [31]:
import base64

with open('file.crl', 'wb') as f:
    f.write(df.iloc[30059].content)

In [30]:
df.iloc[30059]

uri            rsync://rpki.ripe.net/repository/DEFAULT/5e/79...
hash                                                        None
content        b'0\x83\x0bz\xe20\x83\x0by\xc9\x02\x01\x010\r\...
h_content      ca240c09d41373202849d2312865561d2358c4ae2aa5e9...
num_slashes                                                    9
extension                                                    crl
size                                                      752359
Name: 30059, dtype: object